# Data input structure

## Each row represents
- One **Brand**
- In one **Country**
- At one **Month** (Year + Month)

---

## Inputs to the model (`X`)
Each row must provide:

1. **Time features**
   - `time_idx` (int): monotonic month index (0, 1, 2, …).
   - `month_sin` (float): `sin(2π * month/12)`.
   - `month_cos` (float): `cos(2π * month/12)`.

2. **Categorical features**
   - `country_id` (int): encoded Country.
   - `brand_id` (int): encoded Brand.

3. **Controllable features (vector)**
   - Shape: `(n_controls,)`.
   - Example (if n_controls=10):
     `[promotions_index, media_invest, discount_pct, inflation, temperature_c, holiday_flag, Meaning, Difference, Salience, Premium]`
   - Values must be standardized (mean=0, std=1).
   - At **forecast time**: feed a vector of zeros.

---

## Extra data (not part of model inputs but needed in training)
- `group_id` (int): unique ID for each (Country × Date).  
  - All brands in the same Country+Date share the same `group_id`.  
  - Used in loss to apply softmax within groups.

- `y_true` (float): actual Power percentage for that row (0–100).  
  - During training: available.  
  - During forecast: unknown (to be predicted).

---

## Tensor shapes during training
- `time_idx`: `(batch, 1)`
- `month_sin`: `(batch, 1)`
- `month_cos`: `(batch, 1)`
- `country_id`: `(batch, 1)`
- `brand_id`: `(batch, 1)`
- `controls`: `(batch, n_controls)`
- `y_true`: `(batch,)`
- `group_id`: `(batch,)`

---

## Model outputs
- `logits`: `(batch, 1)` — unnormalized additive scores per row.
- `ctrl_out`: `(batch, 1)` — controllable contribution (for regularization).

---

## Loss function
- Uses `(logits, group_id)` to compute **softmax per group** (so all brands in same Country×Date sum to 1).
- Compares predicted fractions × 100 to `y_true` (MSE).
- Adds small L2 penalty on `ctrl_out`.


In [1]:
import pandas as pd
import numpy as np
import pickle as pkl
import warnings
warnings.filterwarnings('ignore')

In [2]:
input_path = "C:/Users/40107904/OneDrive - Anheuser-Busch InBev/ABI/WORK/hackathon_power/hackathon_lt_equity/data"

In [3]:
country_list =[
    "653b4e6d12",
    "a19bbfeafb",
    "aa30935544"
]

# BG Data

In [4]:
# import files for bg data
bg_data = pd.read_csv(f"{input_path}/lte_participants_data/BG_Data_Hackathon_Train_masked.csv")

bg_data = bg_data.drop(columns=['meaning', 'difference', 'salience', 'premium'])
bg_data

,year,quarter,country,brand,power
0,2022,Qtr1,653b4e6d12,de903bdf13,3.075272
1,2022,Qtr2,653b4e6d12,de903bdf13,3.247745
2,2022,Qtr3,653b4e6d12,de903bdf13,3.159509
3,2022,Qtr4,653b4e6d12,de903bdf13,3.428511
4,2023,Qtr1,653b4e6d12,de903bdf13,3.590078
...,...,...,...,...,...
1617,2023,Qtr2,aa30935544,780f2d8e64,2.199271
1618,2023,Qtr3,aa30935544,780f2d8e64,2.128657
1619,2023,Qtr4,aa30935544,780f2d8e64,2.144927
1620,2024,Qtr1,aa30935544,780f2d8e64,1.894465


In [5]:
bg_data_backup = bg_data.copy()

# Weather data

In [6]:
# import files for weather data
w1 = pd.read_csv(f"{input_path}/lte_participants_data/653b4e6d12 WT DATA/weather_data_agg_wkly_delivered_masked.csv")
w1['week_date'] = pd.to_datetime(w1['week_date'])
w1['year'] = w1['week_date'].dt.year
w1['quarter'] = w1['week_date'].dt.quarter
w1['country'] = '653b4e6d12'
w1 = w1[['country', 'year', 'quarter', 'avg_temp', 'avg_max_temp', 'avg_prcp']]
w1 = w1.groupby(['country', 'year', 'quarter']).agg({'avg_temp':'mean', 'avg_max_temp':'mean', 'avg_prcp':'mean'}).reset_index()
w1

,country,year,quarter,avg_temp,avg_max_temp,avg_prcp
0,653b4e6d12,2019,1,78.180110,84.755495,0.211868
1,653b4e6d12,2019,2,74.141868,80.567473,0.080000
2,653b4e6d12,2019,3,73.021648,81.453846,0.023956
3,653b4e6d12,2019,4,77.489231,84.913187,0.143516
4,653b4e6d12,2020,1,77.419121,83.528352,0.224615
5,653b4e6d12,2020,2,72.663846,79.489780,0.075714
6,653b4e6d12,2020,3,73.406484,82.026923,0.027363
7,653b4e6d12,2020,4,77.680989,85.380330,0.139231
8,653b4e6d12,2021,1,77.781538,84.295165,0.207363
9,653b4e6d12,2021,2,72.650989,79.905055,0.054286


In [7]:
w2 = pd.read_csv(f"{input_path}/lte_participants_data/a19bbfeafb WT DATA/weather_masked.csv")
w2['week_date'] = pd.to_datetime(w2['week_date'])
w2['year'] = w2['week_date'].dt.year
w2['quarter'] = w2['week_date'].dt.quarter
w2['country'] = 'a19bbfeafb'
w2 = w2[['country', 'year', 'quarter', 'avg_temp', 'avg_max_temp', 'avg_prcp']]
w2 = w2.groupby(['country', 'year', 'quarter']).agg({'avg_temp':'mean', 'avg_max_temp':'mean', 'avg_prcp':'mean'}).reset_index()
w2

,country,year,quarter,avg_temp,avg_max_temp,avg_prcp
0,a19bbfeafb,2021,1,69.090998,76.691118,0.225254
1,a19bbfeafb,2021,2,70.203470,77.051292,0.261141
2,a19bbfeafb,2021,3,69.727547,77.027274,0.225200
3,a19bbfeafb,2021,4,69.751659,77.108359,0.244904
4,a19bbfeafb,2022,1,69.513185,77.672698,0.126829
5,a19bbfeafb,2022,2,70.073785,77.437205,0.251173
6,a19bbfeafb,2022,3,69.710111,77.775484,0.222373
7,a19bbfeafb,2022,4,68.838522,76.390119,0.195500
8,a19bbfeafb,2023,1,68.962468,76.989979,0.141255
9,a19bbfeafb,2023,2,70.773286,78.502112,0.153427


In [8]:
w3 = pd.read_csv(f"{input_path}/lte_participants_data/aa30935544 WT DATA/weather_masked.csv")
w3['week_date'] = pd.to_datetime(w3['week_date'])
w3['year'] = w3['week_date'].dt.year
w3['quarter'] = w3['week_date'].dt.quarter
w3['country'] = 'aa30935544'
w3 = w3[['country', 'year', 'quarter', 'avg_temp', 'avg_max_temp', 'avg_prcp']]
w3 = w3.groupby(['country', 'year', 'quarter']).agg({'avg_temp':'mean', 'avg_max_temp':'mean', 'avg_prcp':'mean'}).reset_index()
w3

,country,year,quarter,avg_temp,avg_max_temp,avg_prcp
0,aa30935544,2017,1,43.654769,53.363260,0.056913
1,aa30935544,2017,2,64.377881,75.132228,0.104445
2,aa30935544,2017,3,73.684973,84.362517,0.087335
3,aa30935544,2017,4,48.457821,58.637396,0.054900
4,aa30935544,2018,1,40.411747,50.361686,0.067542
5,aa30935544,2018,2,65.608349,76.624378,0.086045
6,aa30935544,2018,3,75.190788,85.410347,0.107520
7,aa30935544,2018,4,47.595915,56.451243,0.094343
8,aa30935544,2019,1,39.257131,48.588041,0.069637
9,aa30935544,2019,2,64.772847,75.275990,0.105966


In [9]:
weather_data = pd.concat([w1, w2, w3], ignore_index=True)
weather_data['quarter'] = weather_data['quarter'].apply(lambda x: 'Qtr' + str(x))
weather_data

,country,year,quarter,avg_temp,avg_max_temp,avg_prcp
0,653b4e6d12,2019,Qtr1,78.180110,84.755495,0.211868
1,653b4e6d12,2019,Qtr2,74.141868,80.567473,0.080000
2,653b4e6d12,2019,Qtr3,73.021648,81.453846,0.023956
3,653b4e6d12,2019,Qtr4,77.489231,84.913187,0.143516
4,653b4e6d12,2020,Qtr1,77.419121,83.528352,0.224615
...,...,...,...,...,...,...
72,aa30935544,2024,Qtr1,43.614048,53.130733,0.112526
73,aa30935544,2024,Qtr2,66.907023,77.792452,0.118285
74,aa30935544,2024,Qtr3,75.569596,86.260089,0.113382
75,aa30935544,2024,Qtr4,51.354994,61.342618,0.076171


# Macroeco data

In [10]:
# # import files for macroeconomic data
# macro_data = pd.read_csv(f"{input_path}/mroi/brazil_macroeconomics_data.csv")

# macro_data['timedesc'] = pd.to_datetime(macro_data['timedesc'], format='%Y-%m-%d')
# macro_data['year'] = macro_data['timedesc'].dt.year
# macro_data['month'] = macro_data['timedesc'].dt.month
# macro_data = macro_data.drop(columns=['timedesc'])
# macro_data = macro_data.groupby(['country','year', 'month']).mean().reset_index()

# macro_data

# Media data

In [11]:
media_mapping = r'C:\Users\40107904\OneDrive - Anheuser-Busch InBev\ABI\WORK\hackathon_power\hackathon_lt_equity\created_data\Mapping.csv'
media_mapping = pd.read_csv(media_mapping)
media_mapping

,Option,New Channel Group
0,corona100,all_other_marketing
1,contracts exclusivity,contracts
2,contracts experiential,contracts
3,experiential,all_other_marketing
4,influencers,Other Digital Media
...,...,...
155,vehicle_tvr_programmatic_hulu,Other Digital Media
156,vehicle_tvr_social,Other Digital Media
157,vehicle_tvr_social_meta,Other Digital Media
158,vehicle_tvr_social_snapchat,Other Digital Media


In [12]:
# import files for media data
media_data = None

country_1_media_data = pd.read_csv(f"{input_path}/lte_participants_data/653b4e6d12 WT DATA/investment_masked.csv")
media_data = country_1_media_data if media_data is None else pd.concat([media_data, country_1_media_data], axis=0)
media_data['country'] = '653b4e6d12'
media_data['week_date'] = pd.to_datetime(media_data['week_date'], format='%Y-%m-%d')
media_data['year'] = media_data['week_date'].dt.year
media_data['quarter'] = media_data['week_date'].dt.quarter
media_data = media_data.drop(columns=['week_date'])
# media_data['quarter'] = media_data['quarter'].apply(lambda x: 'Qtr'+str((x-1)//3 + 1))
media_data = media_data.groupby(['country', 'brand', 'year', 'quarter']).sum().reset_index()
media_data = media_data.drop(columns=['channel', 'region'])

vehicle_cols_1 = [col for col in media_data.columns if col not in ['country', 'brand', 'year', 'quarter']]
media_data = media_data.melt(id_vars=['country', 'brand', 'year', 'quarter'], value_vars=vehicle_cols_1, var_name='vehicle', value_name='investment')

media_data

,country,brand,year,quarter,vehicle,investment
0,653b4e6d12,05e5aaf0f4,2019,1,corona100,0.000000e+00
1,653b4e6d12,05e5aaf0f4,2019,2,corona100,0.000000e+00
2,653b4e6d12,05e5aaf0f4,2019,3,corona100,0.000000e+00
3,653b4e6d12,05e5aaf0f4,2019,4,corona100,0.000000e+00
4,653b4e6d12,05e5aaf0f4,2020,1,corona100,0.000000e+00
...,...,...,...,...,...,...
4699,653b4e6d12,fc6df32a5f,2024,2,youtube,2.105999e+06
4700,653b4e6d12,fc6df32a5f,2024,3,youtube,4.072333e+05
4701,653b4e6d12,fc6df32a5f,2024,4,youtube,1.826896e+06
4702,653b4e6d12,fc6df32a5f,2025,1,youtube,1.244025e+06


In [13]:
country_2_media_data = pd.read_csv(f"{input_path}/lte_participants_data/a19bbfeafb WT DATA/investment_masked.csv")
country_2_media_data['country'] = 'a19bbfeafb'
country_2_media_data['week_date'] = pd.to_datetime(country_2_media_data['week_date'], format='%Y-%m-%d')
country_2_media_data['year'] = country_2_media_data['week_date'].dt.year
country_2_media_data['month'] = country_2_media_data['week_date'].dt.month
country_2_media_data = country_2_media_data.drop(columns=['week_date'])
country_2_media_data['quarter'] = country_2_media_data['month'].apply(lambda x: int((x-1)//3 + 1))
country_2_media_data = country_2_media_data.groupby(['country', 'brand', 'year', 'month', 'quarter']).sum().reset_index()
country_2_media_data = country_2_media_data.drop(columns=['month', 'channel', 'region'])

vehicle_cols_2 = [col for col in country_2_media_data.columns if col not in ['country', 'brand', 'year', 'quarter']]
country_2_media_data = country_2_media_data.melt(id_vars=['country', 'brand', 'year', 'quarter'], value_vars=vehicle_cols_2, var_name='vehicle', value_name='investment')

country_2_media_data
media_data = country_2_media_data if media_data is None else pd.concat([media_data, country_2_media_data], axis=0)
media_data

,country,brand,year,quarter,vehicle,investment
0,653b4e6d12,05e5aaf0f4,2019,1,corona100,0.000000e+00
1,653b4e6d12,05e5aaf0f4,2019,2,corona100,0.000000e+00
2,653b4e6d12,05e5aaf0f4,2019,3,corona100,0.000000e+00
3,653b4e6d12,05e5aaf0f4,2019,4,corona100,0.000000e+00
4,653b4e6d12,05e5aaf0f4,2020,1,corona100,0.000000e+00
...,...,...,...,...,...,...
11173,a19bbfeafb,fdb6b2e750,2025,1,youtube,7.156230e+07
11174,a19bbfeafb,fdb6b2e750,2025,1,youtube,1.269977e+08
11175,a19bbfeafb,fdb6b2e750,2025,2,youtube,1.122087e+08
11176,a19bbfeafb,fdb6b2e750,2025,2,youtube,6.643512e+07


In [14]:
country_3_media_data = pd.read_csv(f"{input_path}/lte_participants_data/aa30935544 WT DATA/investment_masked.csv")
country_3_media_data['country'] = 'aa30935544'
country_3_media_data = country_3_media_data.loc[country_3_media_data['actuals_flag'] == 1]
country_3_media_data['week_date'] = pd.to_datetime(country_3_media_data['week_date'])
country_3_media_data['year'] = country_3_media_data['week_date'].dt.year
country_3_media_data['quarter'] = country_3_media_data['week_date'].dt.quarter
country_3_media_data = country_3_media_data.drop(columns=['week_date'])
# country_3_media_data['quarter'] = country_3_media_data['month'].apply(lambda x: 'Qtr'+str((x-1)//3 + 1))
country_3_media_data = country_3_media_data.groupby(['country', 'brand', 'year', 'quarter']).sum().reset_index()
country_3_media_data = country_3_media_data.drop(columns=[ 'state_code', 'actuals_flag'])

vehicle_cols_3 = [col for col in country_3_media_data.columns if col not in ['country', 'brand', 'year', 'quarter']]
country_3_media_data = country_3_media_data.melt(id_vars=['country', 'brand', 'year', 'quarter'], value_vars=vehicle_cols_3, var_name='vehicle', value_name='investment')

media_data = country_3_media_data if media_data is None else pd.concat([media_data, country_3_media_data], axis=0)
media_data

,country,brand,year,quarter,vehicle,investment
0,653b4e6d12,05e5aaf0f4,2019,1,corona100,0.0
1,653b4e6d12,05e5aaf0f4,2019,2,corona100,0.0
2,653b4e6d12,05e5aaf0f4,2019,3,corona100,0.0
3,653b4e6d12,05e5aaf0f4,2019,4,corona100,0.0
4,653b4e6d12,05e5aaf0f4,2020,1,corona100,0.0
...,...,...,...,...,...,...
32786,aa30935544,f5be73e92d,2023,4,vehicle_tvr_social_youtube,0.0
32787,aa30935544,f5be73e92d,2024,1,vehicle_tvr_social_youtube,0.0
32788,aa30935544,f5be73e92d,2024,2,vehicle_tvr_social_youtube,0.0
32789,aa30935544,f5be73e92d,2024,3,vehicle_tvr_social_youtube,0.0


In [15]:
media_data = media_data.merge(media_mapping, how='left', left_on=['vehicle'], right_on=['Option'])
media_data

,country,brand,year,quarter,vehicle,investment,Option,New Channel Group
0,653b4e6d12,05e5aaf0f4,2019,1,corona100,0.0,corona100,all_other_marketing
1,653b4e6d12,05e5aaf0f4,2019,2,corona100,0.0,corona100,all_other_marketing
2,653b4e6d12,05e5aaf0f4,2019,3,corona100,0.0,corona100,all_other_marketing
3,653b4e6d12,05e5aaf0f4,2019,4,corona100,0.0,corona100,all_other_marketing
4,653b4e6d12,05e5aaf0f4,2020,1,corona100,0.0,corona100,all_other_marketing
...,...,...,...,...,...,...,...,...
48668,aa30935544,f5be73e92d,2023,4,vehicle_tvr_social_youtube,0.0,vehicle_tvr_social_youtube,Other Digital Media
48669,aa30935544,f5be73e92d,2024,1,vehicle_tvr_social_youtube,0.0,vehicle_tvr_social_youtube,Other Digital Media
48670,aa30935544,f5be73e92d,2024,2,vehicle_tvr_social_youtube,0.0,vehicle_tvr_social_youtube,Other Digital Media
48671,aa30935544,f5be73e92d,2024,3,vehicle_tvr_social_youtube,0.0,vehicle_tvr_social_youtube,Other Digital Media


In [16]:
media_data = media_data.groupby(['country', 'brand', 'year', 'quarter', 'New Channel Group']).agg({'investment':'sum'}).reset_index()
media_data

,country,brand,year,quarter,New Channel Group,investment
0,653b4e6d12,05e5aaf0f4,2019,1,OOH,0.000000e+00
1,653b4e6d12,05e5aaf0f4,2019,1,Other Digital Media,0.000000e+00
2,653b4e6d12,05e5aaf0f4,2019,1,Other Traditional Mktg,1.974493e+05
3,653b4e6d12,05e5aaf0f4,2019,1,Radio,0.000000e+00
4,653b4e6d12,05e5aaf0f4,2019,1,Social Media,0.000000e+00
...,...,...,...,...,...,...
5061,aa30935544,f5be73e92d,2024,4,Radio,0.000000e+00
5062,aa30935544,f5be73e92d,2024,4,Social Media,1.867046e+04
5063,aa30935544,f5be73e92d,2024,4,TV,0.000000e+00
5064,aa30935544,f5be73e92d,2024,4,Trade,3.394098e+04


In [17]:
media_data = media_data.rename(columns={'New Channel Group':'vehicle'})
media_data = media_data.pivot_table(index=['country', 'brand', 'year', 'quarter'], columns='vehicle', values='investment', fill_value=0).reset_index()
media_data

vehicle,country,brand,year,quarter,OOH,Other Digital Media,Other Traditional Mktg,Radio,Social Media,TV,Trade,all_other_marketing,contracts
0,653b4e6d12,05e5aaf0f4,2019,1,0.000000e+00,0.000000e+00,197449.289806,0.000000,0.000000,0.0,1.362509e+06,1.122270e+05,1.466854e+06
1,653b4e6d12,05e5aaf0f4,2019,2,0.000000e+00,0.000000e+00,94345.289962,0.000000,0.000000,0.0,3.851498e+06,5.197700e+04,1.524613e+06
2,653b4e6d12,05e5aaf0f4,2019,3,0.000000e+00,0.000000e+00,22061.461188,0.000000,0.000000,0.0,5.489670e+05,5.038300e+04,1.617431e+06
3,653b4e6d12,05e5aaf0f4,2019,4,0.000000e+00,0.000000e+00,138859.671750,0.000000,0.000000,0.0,2.411174e+05,5.038300e+04,1.500704e+06
4,653b4e6d12,05e5aaf0f4,2020,1,0.000000e+00,0.000000e+00,401987.066491,0.000000,0.000000,0.0,1.281093e+05,0.000000e+00,1.411434e+06
...,...,...,...,...,...,...,...,...,...,...,...,...,...
624,aa30935544,f5be73e92d,2023,4,4.518178e+04,3.535172e+05,0.000000,6735.710013,18095.665094,0.0,1.851762e+04,5.012171e+06,0.000000e+00
625,aa30935544,f5be73e92d,2024,1,2.037981e+05,1.174271e+06,3657.604000,0.000000,336207.088907,0.0,7.707105e+04,5.488942e+06,0.000000e+00
626,aa30935544,f5be73e92d,2024,2,2.102249e+06,3.386695e+06,1164.426000,0.000000,857760.889893,0.0,9.809439e+04,6.393300e+06,0.000000e+00
627,aa30935544,f5be73e92d,2024,3,2.087783e+06,2.853275e+06,4318.281596,0.000000,257942.283436,0.0,1.085666e+05,5.869649e+06,0.000000e+00


In [18]:
media_data['total_spend'] = media_data[[col for col in media_data.columns if col not in ['country', 'brand', 'year', 'quarter']]].sum(axis=1)
for col in media_data.columns:
    if col not in ['country', 'brand', 'year', 'quarter', 'total_spend']:
        media_data[col] = media_data[col] / media_data['total_spend']
media_data['total_spend'] = media_data['total_spend'] / media_data['total_spend'].max()
media_data

vehicle,country,brand,year,quarter,OOH,Other Digital Media,Other Traditional Mktg,Radio,Social Media,TV,Trade,all_other_marketing,contracts,total_spend
0,653b4e6d12,05e5aaf0f4,2019,1,0.000000,0.000000,0.062901,0.000000,0.000000,0.0,0.434053,0.035752,0.467294,0.000094
1,653b4e6d12,05e5aaf0f4,2019,2,0.000000,0.000000,0.017084,0.000000,0.000000,0.0,0.697428,0.009412,0.276076,0.000166
2,653b4e6d12,05e5aaf0f4,2019,3,0.000000,0.000000,0.009854,0.000000,0.000000,0.0,0.245201,0.022504,0.722441,0.000067
3,653b4e6d12,05e5aaf0f4,2019,4,0.000000,0.000000,0.071908,0.000000,0.000000,0.0,0.124862,0.026091,0.777138,0.000058
4,653b4e6d12,05e5aaf0f4,2020,1,0.000000,0.000000,0.207046,0.000000,0.000000,0.0,0.065984,0.000000,0.726970,0.000058
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
624,aa30935544,f5be73e92d,2023,4,0.008284,0.064815,0.000000,0.001235,0.003318,0.0,0.003395,0.918953,0.000000,0.000164
625,aa30935544,f5be73e92d,2024,1,0.027979,0.161214,0.000502,0.000000,0.046157,0.0,0.010581,0.753567,0.000000,0.000218
626,aa30935544,f5be73e92d,2024,2,0.163736,0.263776,0.000091,0.000000,0.066808,0.0,0.007640,0.497949,0.000000,0.000385
627,aa30935544,f5be73e92d,2024,3,0.186717,0.255177,0.000386,0.000000,0.023069,0.0,0.009709,0.524941,0.000000,0.000335


In [19]:
media_data['quarter'] = media_data['quarter'].apply(lambda x: 'Qtr' + str(x))

In [20]:
print(media_data['country'].value_counts())

country
aa30935544    271
653b4e6d12    196
a19bbfeafb    162
Name: count, dtype: int64


In [21]:
media_data

vehicle,country,brand,year,quarter,OOH,Other Digital Media,Other Traditional Mktg,Radio,Social Media,TV,Trade,all_other_marketing,contracts,total_spend
0,653b4e6d12,05e5aaf0f4,2019,Qtr1,0.000000,0.000000,0.062901,0.000000,0.000000,0.0,0.434053,0.035752,0.467294,0.000094
1,653b4e6d12,05e5aaf0f4,2019,Qtr2,0.000000,0.000000,0.017084,0.000000,0.000000,0.0,0.697428,0.009412,0.276076,0.000166
2,653b4e6d12,05e5aaf0f4,2019,Qtr3,0.000000,0.000000,0.009854,0.000000,0.000000,0.0,0.245201,0.022504,0.722441,0.000067
3,653b4e6d12,05e5aaf0f4,2019,Qtr4,0.000000,0.000000,0.071908,0.000000,0.000000,0.0,0.124862,0.026091,0.777138,0.000058
4,653b4e6d12,05e5aaf0f4,2020,Qtr1,0.000000,0.000000,0.207046,0.000000,0.000000,0.0,0.065984,0.000000,0.726970,0.000058
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
624,aa30935544,f5be73e92d,2023,Qtr4,0.008284,0.064815,0.000000,0.001235,0.003318,0.0,0.003395,0.918953,0.000000,0.000164
625,aa30935544,f5be73e92d,2024,Qtr1,0.027979,0.161214,0.000502,0.000000,0.046157,0.0,0.010581,0.753567,0.000000,0.000218
626,aa30935544,f5be73e92d,2024,Qtr2,0.163736,0.263776,0.000091,0.000000,0.066808,0.0,0.007640,0.497949,0.000000,0.000385
627,aa30935544,f5be73e92d,2024,Qtr3,0.186717,0.255177,0.000386,0.000000,0.023069,0.0,0.009709,0.524941,0.000000,0.000335


# Neilsen

In [22]:
# import files for pos data
c1=pd.read_csv(f'{input_path}/lte_participants_data/653b4e6d12 WT DATA/sellout_2020_2025Q2_raw_from_excel_files_masked.csv')
c2=pd.read_csv(f'{input_path}/lte_participants_data/a19bbfeafb WT DATA/col_sellout_raw_data_masked.csv')
c3=pd.read_csv(f'{input_path}/lte_participants_data/aa30935544 WT DATA/revenue_masked.csv')

In [23]:
df1 = c1
df2 = c2
df3 = c3

# -----------------
# Process Data 1
# -----------------
# df1["date"] = pd.to_datetime(df1["date"])
df1 = pd.DataFrame({
    "year": df1["year"],
    "quarter": ((df1["month number"] - 1)//3 + 1).apply(lambda x: f"Qtr{x}"),
    "country": "653b4e6d12",   # fixed for all rows in data1
    "brand": df1["brand"],
    "sales_vol": df1["normalized sales (in hectoliters)"],
    "sales": df1["normalized sales value"]
})
# df1["price"] = df1["sales"] / df1["sales_vol"]
 
# -----------------
# Process Data 2
# -----------------
df2["date"] = pd.to_datetime(df2["date"])
df2 = pd.DataFrame({
    "year": df2["date"].dt.year,
    "quarter": df2["date"].dt.quarter.apply(lambda x: f"Qtr{x}"),
    "country": "a19bbfeafb",   # fixed for all rows in data2
    "brand": df2["brand"],
    "sales_vol": df2["sales_hectoliters"],
    "sales": df2["000_sales_value"]
})
# df2["price"] = df2["sales"] / df2["sales_vol"]
 
# -----------------
# Process Data 3
# -----------------
df3['week_date'] = pd.to_datetime(df3['week_date'])
df3 = pd.DataFrame({
    "year": df3["week_date"].dt.year,
    "quarter": df3["week_date"].dt.quarter.apply(lambda x: f"Qtr{x}"),
    "country": "aa30935544",   # fixed for all rows in data3
    "brand": df3["brand"],
    "sales_vol": df3["volume"],
    "sales": df3["sales"]
})
 

In [24]:
def clean_and_report(df, cols, label):
    print(f"\n===== {label} =====")
    for col in cols:
        series = df[col].astype(str).str.strip()

        # Count real NaNs before conversion
        real_nans_before = df[col].isna().sum()

        # Before cleaning: attempt numeric conversion
        before_numeric = pd.to_numeric(series, errors="coerce")
        before_problem = before_numeric.isna() & series.notna()

        print(f"\nColumn: {col}")
        print(f"  Total rows: {len(series)}")
        print(f"  Real NaNs BEFORE cleaning: {real_nans_before}")
        print(f"  Problematic (non-numeric strings) BEFORE cleaning: {before_problem.sum()}")

        # Cleaning: remove commas, spaces, and replace string nans
        series_cleaned = (
            series
            .str.replace(",", "", regex=False)
            .str.replace(" ", "", regex=False)
            .replace({"nan": None, "NaN": None, "NULL": None, "None": None})
        )

        # After cleaning
        after_numeric = pd.to_numeric(series_cleaned, errors="coerce")
        after_problem = after_numeric.isna() & series_cleaned.notna()

        print(f"  Problematic AFTER cleaning: {after_problem.sum()}")
        print(f"  Real NaNs AFTER cleaning: {after_numeric.isna().sum()}")

        # Replace in df with cleaned numeric
        df[col] = after_numeric.astype(float)

    return df


# -----------------
# Run for Data 1
# -----------------
df1 = clean_and_report(
    df1,
    ["sales_vol", "sales"],
    "Data1"
)

# -----------------
# Run for Data 2
# -----------------
df2 = clean_and_report(
    df2,
    ["sales_vol", "sales"],
    "Data2"
)

# -----------------
# Run for Data 3
# -----------------
df3 = clean_and_report(
    df3,
    ["sales_vol", "sales"],
    "Data3"
)



===== Data1 =====

Column: sales_vol
  Total rows: 366403
  Real NaNs BEFORE cleaning: 0
  Problematic (non-numeric strings) BEFORE cleaning: 23190
  Problematic AFTER cleaning: 0
  Real NaNs AFTER cleaning: 0

Column: sales
  Total rows: 366403
  Real NaNs BEFORE cleaning: 0
  Problematic (non-numeric strings) BEFORE cleaning: 80749
  Problematic AFTER cleaning: 0
  Real NaNs AFTER cleaning: 0

===== Data2 =====

Column: sales_vol
  Total rows: 748459
  Real NaNs BEFORE cleaning: 58380
  Problematic (non-numeric strings) BEFORE cleaning: 58380
  Problematic AFTER cleaning: 0
  Real NaNs AFTER cleaning: 58380

Column: sales
  Total rows: 748459
  Real NaNs BEFORE cleaning: 58380
  Problematic (non-numeric strings) BEFORE cleaning: 58380
  Problematic AFTER cleaning: 0
  Real NaNs AFTER cleaning: 58380

===== Data3 =====

Column: sales_vol
  Total rows: 1048575
  Real NaNs BEFORE cleaning: 0
  Problematic (non-numeric strings) BEFORE cleaning: 0
  Problematic AFTER cleaning: 0
  Real N

In [25]:
# List of numeric columns to check
numeric_cols = ["sales_vol", "sales"]

# Drop rows with NaN in any of those columns
df1 = df1.dropna(subset=numeric_cols)
df2 = df2.dropna(subset=numeric_cols)
df3 = df3.dropna(subset=numeric_cols)

In [26]:
#-----------------
#Combine all 3 datasets
#-----------------
final_df = pd.concat([df1, df2, df3], ignore_index=True)
final_df.fillna(0, inplace=True)

#-----------------
#Aggregate
#-----------------
agg_df = (
    final_df.groupby(["year","quarter","country","brand"], as_index=False)
    .agg({
        "sales_vol": "sum",
        "sales": "sum"
    })
)
agg_df["price"] = agg_df["sales"] / agg_df["sales_vol"]

# Save outputs if needed
# final_df.to_csv("combined_raw.csv", index=False)         # row-level with price
# agg_df.to_csv("combined_aggregated.csv", index=False)    # aggregated with price

In [27]:
pos_data = agg_df
pos_data

,year,quarter,country,brand,sales_vol,sales,price
0,2017,Qtr2,aa30935544,1632ec491e,4.689145e+06,1.126995e+09,240.341372
1,2017,Qtr2,aa30935544,1c2d8e1785,3.168129e+05,5.434252e+07,171.528746
2,2017,Qtr2,aa30935544,2601ae05f5,1.015178e+06,3.034071e+08,298.870694
3,2017,Qtr2,aa30935544,2654678443,1.222992e+05,4.055373e+07,331.594425
4,2017,Qtr2,aa30935544,52be03db3b,9.000579e+05,1.637795e+08,181.965520
...,...,...,...,...,...,...,...
3141,2025,Qtr2,a19bbfeafb,967e3b26f8,3.277391e+07,2.817926e+10,859.807703
3142,2025,Qtr2,a19bbfeafb,a0bc54a71d,1.797174e+06,1.982901e+09,1103.343966
3143,2025,Qtr2,a19bbfeafb,b6a118bf97,2.030168e+07,1.843063e+10,907.837662
3144,2025,Qtr2,a19bbfeafb,cd46a210c6,4.349904e+07,3.862489e+10,887.948071


# Segment

In [28]:
seg_map = pd.read_csv("../created_data/seg_map.csv")
seg_map

,brand,category,country
0,fc6df32a5f,core,653b4e6d12
1,0ec35f4c94,core,653b4e6d12
2,2441ad3052,core+,653b4e6d12
3,7690b7ccb9,super premium,653b4e6d12
4,05e5aaf0f4,premium,653b4e6d12
5,875962380f,core,653b4e6d12
6,d31214cc07,premium,653b4e6d12
7,f9053fad96,super premium,653b4e6d12
8,45b1ae2725,super premium,a19bbfeafb
9,fdb6b2e750,super premium,a19bbfeafb


# Concantenate

In [95]:
final_df = bg_data.copy()
final_df = final_df.merge(weather_data, on=['country', 'year', 'quarter'], how='left')
# final_df = final_df.merge(macro_data, on=['country', 'year', 'month'], how='left')
final_df = final_df.merge(media_data, on=['country', 'brand', 'year', 'quarter'], how='left')
final_df = final_df.merge(pos_data, on=['country', 'brand', 'year', 'quarter'], how='left')
final_df = final_df.merge(seg_map, on=['country', 'brand'], how='left')
final_df

,year,quarter,country,brand,power,avg_temp,avg_max_temp,avg_prcp,OOH,Other Digital Media,...,Social Media,TV,Trade,all_other_marketing,contracts,total_spend,sales_vol,sales,price,category
0,2022,Qtr1,653b4e6d12,de903bdf13,3.075272,77.645165,84.032637,0.223846,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,1.149127e+06,1.088902e+09,947.590722,NaN
1,2022,Qtr2,653b4e6d12,de903bdf13,3.247745,72.486264,79.484396,0.078571,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,1.110635e+06,1.102294e+09,992.490012,NaN
2,2022,Qtr3,653b4e6d12,de903bdf13,3.159509,72.453407,80.874176,0.031209,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,1.185501e+06,1.210153e+09,1020.794937,NaN
3,2022,Qtr4,653b4e6d12,de903bdf13,3.428511,75.811531,83.187653,0.157347,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,1.482052e+06,1.550552e+09,1046.219460,NaN
4,2023,Qtr1,653b4e6d12,de903bdf13,3.590078,77.510119,83.735238,0.244524,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,1.534081e+06,1.691719e+09,1102.757563,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1617,2023,Qtr2,aa30935544,780f2d8e64,2.199271,63.150540,74.317540,0.086131,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1618,2023,Qtr3,aa30935544,780f2d8e64,2.128657,75.273772,85.921747,0.069734,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1619,2023,Qtr4,aa30935544,780f2d8e64,2.144927,50.513863,60.088509,0.083362,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1620,2024,Qtr1,aa30935544,780f2d8e64,1.894465,43.614048,53.130733,0.112526,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [96]:
abi_brands = final_df.loc[final_df['total_spend'].isna() == False]
abi_brands

,year,quarter,country,brand,power,avg_temp,avg_max_temp,avg_prcp,OOH,Other Digital Media,...,Social Media,TV,Trade,all_other_marketing,contracts,total_spend,sales_vol,sales,price,category
20,2022,Qtr1,653b4e6d12,0ec35f4c94,3.238217,77.645165,84.032637,0.223846,0.000000,0.000000,...,0.000000,0.000000,0.028328,0.003523,0.661457,0.000463,2.709707e+06,2.651019e+09,978.341548,core
21,2022,Qtr2,653b4e6d12,0ec35f4c94,3.469604,72.486264,79.484396,0.078571,0.000000,0.000000,...,0.000000,0.000000,0.023973,0.003688,0.689964,0.000443,2.653983e+06,2.643550e+09,996.069077,core
22,2022,Qtr3,653b4e6d12,0ec35f4c94,3.271975,72.453407,80.874176,0.031209,0.000000,0.000000,...,0.000000,0.000000,0.031824,0.003687,0.653010,0.000443,2.592051e+06,2.654256e+09,1023.998623,core
23,2022,Qtr4,653b4e6d12,0ec35f4c94,3.327583,75.811531,83.187653,0.157347,0.000000,0.000000,...,0.000000,0.000000,0.070517,0.003640,0.695357,0.000449,2.852684e+06,3.015454e+09,1057.058561,core
24,2023,Qtr1,653b4e6d12,0ec35f4c94,3.233185,77.510119,83.735238,0.244524,0.000000,0.000000,...,0.000000,0.000000,0.008660,0.009638,0.607621,0.000463,2.847592e+06,3.119109e+09,1095.349830,core
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1545,2023,Qtr2,aa30935544,6e31ce004a,3.090206,63.150540,74.317540,0.086131,0.126893,0.250256,...,0.035903,0.032608,0.007583,0.545307,0.000000,0.000801,2.619703e+05,1.102412e+08,420.815696,NaN
1546,2023,Qtr3,aa30935544,6e31ce004a,3.179394,75.273772,85.921747,0.069734,0.032186,0.377225,...,0.040891,0.031034,0.003022,0.509737,0.000000,0.000815,2.710217e+05,1.139982e+08,420.624034,NaN
1547,2023,Qtr4,aa30935544,6e31ce004a,2.946812,50.513863,60.088509,0.083362,0.027153,0.132714,...,0.013479,0.000286,0.005646,0.815235,0.000000,0.000614,2.565668e+05,1.080037e+08,420.957452,NaN
1548,2024,Qtr1,aa30935544,6e31ce004a,3.046745,43.614048,53.130733,0.112526,0.096482,0.118127,...,0.030602,0.000000,0.023266,0.729708,0.000000,0.000356,2.063179e+05,8.755758e+07,424.381866,NaN


In [97]:
print(abi_brands.columns.tolist())

['year', 'quarter', 'country', 'brand', 'power', 'avg_temp', 'avg_max_temp', 'avg_prcp', 'OOH', 'Other Digital Media', 'Other Traditional Mktg', 'Radio', 'Social Media', 'TV', 'Trade', 'all_other_marketing', 'contracts', 'total_spend', 'sales_vol', 'sales', 'price', 'category']


In [98]:
competitor_brands = final_df.loc[final_df['total_spend'].isna() == True]
competitor_brands = competitor_brands.groupby(['country', 'year', 'quarter']).agg({
    'power': 'sum',
    'OOH': 'sum',
    'Other Digital Media': 'sum',
    'Other Traditional Mktg': 'sum',
    'TV': 'sum',
    'Radio': 'sum',
    'Social Media': 'sum',
    'Trade': 'sum',
    'all_other_marketing': 'sum',
    'contracts': 'sum',
    'avg_temp':'mean',
    'avg_max_temp':'mean',
    'avg_prcp':'mean',
    'total_spend':'sum',
    'sales_vol':'sum',
    'sales':'sum',
    'price':'mean',
    'category': 'first'
}).reset_index()
competitor_brands['brand'] = 'Competitor'

competitor_brands['avg_temp'] = competitor_brands['avg_temp'].fillna(competitor_brands['avg_temp'].mean())
competitor_brands['avg_max_temp'] = competitor_brands['avg_max_temp'].fillna(competitor_brands['avg_max_temp'].mean())
competitor_brands['avg_prcp'] = competitor_brands['avg_prcp'].fillna(competitor_brands['avg_prcp'].mean())

country_avg_price = competitor_brands.groupby(['country']).agg({'price':'mean'}).reset_index()
competitor_brands = competitor_brands.merge(country_avg_price, on=['country'], suffixes=('', '_avg'))
competitor_brands['price'] = competitor_brands['price'].fillna(competitor_brands['price_avg'])
competitor_brands = competitor_brands.drop(columns=['price_avg'])

competitor_brands['category'] = competitor_brands['category'].fillna('Unknown')
competitor_brands = competitor_brands[competitor_brands['power'] < 99]

competitor_brands

,country,year,quarter,power,OOH,Other Digital Media,Other Traditional Mktg,TV,Radio,Social Media,...,contracts,avg_temp,avg_max_temp,avg_prcp,total_spend,sales_vol,sales,price,category,brand
0,653b4e6d12,2022,Qtr1,57.471137,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,77.645165,84.032637,0.223846,0.0,9.203216e+06,1.053933e+10,1195.737684,Unknown,Competitor
1,653b4e6d12,2022,Qtr2,56.852930,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,72.486264,79.484396,0.078571,0.0,8.412762e+06,1.005602e+10,1214.469891,Unknown,Competitor
2,653b4e6d12,2022,Qtr3,56.427377,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,72.453407,80.874176,0.031209,0.0,8.399370e+06,1.035883e+10,1219.460052,Unknown,Competitor
3,653b4e6d12,2022,Qtr4,55.935113,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,75.811531,83.187653,0.157347,0.0,9.699839e+06,1.231397e+10,1243.778652,Unknown,Competitor
4,653b4e6d12,2023,Qtr1,55.924409,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,77.510119,83.735238,0.244524,0.0,9.634918e+06,1.266336e+10,1277.746577,Unknown,Competitor
5,653b4e6d12,2023,Qtr2,55.910946,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,72.999231,79.679121,0.079670,0.0,8.194774e+06,1.106756e+10,1291.620842,Unknown,Competitor
6,653b4e6d12,2023,Qtr3,56.345919,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,74.640816,83.096429,0.025918,0.0,8.286586e+06,1.130467e+10,1309.372349,Unknown,Competitor
7,653b4e6d12,2023,Qtr4,55.759279,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,79.898352,87.904066,0.096044,0.0,9.454244e+06,1.313423e+10,1349.152485,Unknown,Competitor
8,653b4e6d12,2024,Qtr1,54.865966,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,78.852637,84.962198,0.214505,0.0,9.422144e+06,1.326361e+10,1376.926839,Unknown,Competitor
9,653b4e6d12,2024,Qtr2,54.039295,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,75.213187,82.452308,0.069451,0.0,8.396583e+06,1.194383e+10,1365.110366,Unknown,Competitor


In [99]:
final_df = pd.concat([abi_brands, competitor_brands], ignore_index=True)
final_df

,year,quarter,country,brand,power,avg_temp,avg_max_temp,avg_prcp,OOH,Other Digital Media,...,Social Media,TV,Trade,all_other_marketing,contracts,total_spend,sales_vol,sales,price,category
0,2022,Qtr1,653b4e6d12,0ec35f4c94,3.238217,77.645165,84.032637,0.223846,0.0,0.0,...,0.0,0.0,0.028328,0.003523,0.661457,0.000463,2.709707e+06,2.651019e+09,978.341548,core
1,2022,Qtr2,653b4e6d12,0ec35f4c94,3.469604,72.486264,79.484396,0.078571,0.0,0.0,...,0.0,0.0,0.023973,0.003688,0.689964,0.000443,2.653983e+06,2.643550e+09,996.069077,core
2,2022,Qtr3,653b4e6d12,0ec35f4c94,3.271975,72.453407,80.874176,0.031209,0.0,0.0,...,0.0,0.0,0.031824,0.003687,0.653010,0.000443,2.592051e+06,2.654256e+09,1023.998623,core
3,2022,Qtr4,653b4e6d12,0ec35f4c94,3.327583,75.811531,83.187653,0.157347,0.0,0.0,...,0.0,0.0,0.070517,0.003640,0.695357,0.000449,2.852684e+06,3.015454e+09,1057.058561,core
4,2023,Qtr1,653b4e6d12,0ec35f4c94,3.233185,77.510119,83.735238,0.244524,0.0,0.0,...,0.0,0.0,0.008660,0.009638,0.607621,0.000463,2.847592e+06,3.119109e+09,1095.349830,core
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
365,2023,Qtr2,aa30935544,Competitor,77.313137,63.150540,74.317540,0.086131,0.0,0.0,...,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000e+00,0.000000e+00,286.010838,Unknown
366,2023,Qtr3,aa30935544,Competitor,77.590503,75.273772,85.921747,0.069734,0.0,0.0,...,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000e+00,0.000000e+00,286.010838,Unknown
367,2023,Qtr4,aa30935544,Competitor,77.300686,50.513863,60.088509,0.083362,0.0,0.0,...,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000e+00,0.000000e+00,286.010838,Unknown
368,2024,Qtr1,aa30935544,Competitor,76.953724,43.614048,53.130733,0.112526,0.0,0.0,...,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000e+00,0.000000e+00,286.010838,Unknown


In [100]:
# final_df = final_df.loc[final_df['OOH'].isna() == False]
print(final_df['total_spend'].isna().sum())   

0


In [101]:
# for col in final_df.columns:
#     if final_df[col].dtype in [np.float64, np.float32, np.int64, np.int32]:
#         # fill country's avg for that column
#         country_avg = final_df.groupby(['country'])[col].transform('mean')
#         final_df[col] = final_df[col].fillna(country_avg)
#         # final_df[col] = final_df[col].fillna(0)

# final_df

In [102]:
final_df['category'] = final_df['category'].fillna('Unknown')

In [103]:
print(final_df['country'].value_counts())

country
a19bbfeafb    140
aa30935544    140
653b4e6d12     90
Name: count, dtype: int64


In [104]:
final_df = final_df.fillna(0)
print(final_df.isna().sum())

year                      0
quarter                   0
country                   0
brand                     0
power                     0
avg_temp                  0
avg_max_temp              0
avg_prcp                  0
OOH                       0
Other Digital Media       0
Other Traditional Mktg    0
Radio                     0
Social Media              0
TV                        0
Trade                     0
all_other_marketing       0
contracts                 0
total_spend               0
sales_vol                 0
sales                     0
price                     0
category                  0
dtype: int64


In [105]:
print(final_df['total_spend'].mean())

0.06452868175508679


In [106]:
final_df

,year,quarter,country,brand,power,avg_temp,avg_max_temp,avg_prcp,OOH,Other Digital Media,...,Social Media,TV,Trade,all_other_marketing,contracts,total_spend,sales_vol,sales,price,category
0,2022,Qtr1,653b4e6d12,0ec35f4c94,3.238217,77.645165,84.032637,0.223846,0.0,0.0,...,0.0,0.0,0.028328,0.003523,0.661457,0.000463,2.709707e+06,2.651019e+09,978.341548,core
1,2022,Qtr2,653b4e6d12,0ec35f4c94,3.469604,72.486264,79.484396,0.078571,0.0,0.0,...,0.0,0.0,0.023973,0.003688,0.689964,0.000443,2.653983e+06,2.643550e+09,996.069077,core
2,2022,Qtr3,653b4e6d12,0ec35f4c94,3.271975,72.453407,80.874176,0.031209,0.0,0.0,...,0.0,0.0,0.031824,0.003687,0.653010,0.000443,2.592051e+06,2.654256e+09,1023.998623,core
3,2022,Qtr4,653b4e6d12,0ec35f4c94,3.327583,75.811531,83.187653,0.157347,0.0,0.0,...,0.0,0.0,0.070517,0.003640,0.695357,0.000449,2.852684e+06,3.015454e+09,1057.058561,core
4,2023,Qtr1,653b4e6d12,0ec35f4c94,3.233185,77.510119,83.735238,0.244524,0.0,0.0,...,0.0,0.0,0.008660,0.009638,0.607621,0.000463,2.847592e+06,3.119109e+09,1095.349830,core
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
365,2023,Qtr2,aa30935544,Competitor,77.313137,63.150540,74.317540,0.086131,0.0,0.0,...,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000e+00,0.000000e+00,286.010838,Unknown
366,2023,Qtr3,aa30935544,Competitor,77.590503,75.273772,85.921747,0.069734,0.0,0.0,...,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000e+00,0.000000e+00,286.010838,Unknown
367,2023,Qtr4,aa30935544,Competitor,77.300686,50.513863,60.088509,0.083362,0.0,0.0,...,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000e+00,0.000000e+00,286.010838,Unknown
368,2024,Qtr1,aa30935544,Competitor,76.953724,43.614048,53.130733,0.112526,0.0,0.0,...,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000e+00,0.000000e+00,286.010838,Unknown


# Model Input Creation

In [107]:
final_df['quarter'] = final_df['quarter'].apply(lambda x: int(x.replace('Qtr','')))

In [108]:
final_df_backup = final_df.copy()

In [109]:
df = final_df_backup.copy()

# time_idx column
df = df.sort_values(by=['country', 'year', 'quarter', 'brand']).reset_index(drop=True)
df

df = df.sort_values(['year', 'quarter'])
min_year, min_quarter = df['year'].min(), df['quarter'][df['year'] == df['year'].min()].min()

df['time_idx'] = (df['year'] - min_year) * 4 + (df['quarter'] - min_quarter)

# 3. Cyclical month encoding
df['quarter_sin'] = np.sin(2 * np.pi * df['quarter'] / 4.0).round(6)
df['quarter_cos'] = np.cos(2 * np.pi * df['quarter'] / 4.0).round(6)
df

,year,quarter,country,brand,power,avg_temp,avg_max_temp,avg_prcp,OOH,Other Digital Media,...,all_other_marketing,contracts,total_spend,sales_vol,sales,price,category,time_idx,quarter_sin,quarter_cos
90,2021,1,a19bbfeafb,2aae39f89f,3.894573,69.090998,76.691118,0.225254,0.000000,0.000000,...,0.350894,0.0,0.011453,0.000000e+00,0.000000e+00,0.000000,core,0,1.0,0.0
91,2021,1,a19bbfeafb,45b1ae2725,1.257788,69.090998,76.691118,0.225254,0.000000,0.000000,...,0.141496,0.0,0.043447,0.000000e+00,0.000000e+00,0.000000,super premium,0,1.0,0.0
92,2021,1,a19bbfeafb,53a67488ad,0.794904,69.090998,76.691118,0.225254,0.000000,0.000000,...,0.611176,0.0,0.019529,0.000000e+00,0.000000e+00,0.000000,core,0,1.0,0.0
93,2021,1,a19bbfeafb,85498197e2,11.651194,69.090998,76.691118,0.225254,0.097867,0.132583,...,0.279101,0.0,0.051353,0.000000e+00,0.000000e+00,0.000000,Unknown,0,1.0,0.0
94,2021,1,a19bbfeafb,967e3b26f8,18.567640,69.090998,76.691118,0.225254,0.036453,0.064803,...,0.351557,0.0,0.240463,0.000000e+00,0.000000e+00,0.000000,core,0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
365,2024,2,aa30935544,71e9e5f0c0,1.515613,66.907023,77.792452,0.118285,0.000000,0.000000,...,0.993871,0.0,0.000007,5.361991e+04,1.946623e+07,363.041014,Unknown,13,0.0,-1.0
366,2024,2,aa30935544,96ddb9088d,4.389797,66.907023,77.792452,0.118285,0.000692,0.032690,...,0.857280,0.0,0.001162,1.110928e+06,3.177276e+08,286.001898,Unknown,13,0.0,-1.0
367,2024,2,aa30935544,Competitor,77.046284,66.907023,77.792452,0.118285,0.000000,0.000000,...,0.000000,0.0,0.000000,0.000000e+00,0.000000e+00,286.010838,Unknown,13,0.0,-1.0
368,2024,2,aa30935544,a5f7e77993,1.725634,66.907023,77.792452,0.118285,0.000000,0.000000,...,0.992411,0.0,0.000005,4.939562e+04,1.760712e+07,356.451065,Unknown,13,0.0,-1.0


In [110]:
# from sklearn.preprocessing import StandardScaler
# scaler = StandardScaler()
vehicle_cols = ['OOH', 'Other Digital Media',
       'Other Traditional Mktg', 'Radio', 'Social Media', 'TV', 'Trade',
       'all_other_marketing', 'contracts']

numeric_cols = []
for col in df.select_dtypes(include=['float64', 'int64']).columns:
    if col not in ['year', 'quarter', 'power', 'time_idx'] and col not in vehicle_cols:
        numeric_cols.append(col)
for col in numeric_cols:
    if col in media_data.columns:
        media_data[col] = (media_data[col] - df[col].min()) / df[col].max()
    df[col] = (df[col] - df[col].min()) / df[col].max()
    df[col] = df[col].astype(np.float32)
df

,year,quarter,country,brand,power,avg_temp,avg_max_temp,avg_prcp,OOH,Other Digital Media,...,all_other_marketing,contracts,total_spend,sales_vol,sales,price,category,time_idx,quarter_sin,quarter_cos
90,2021,1,a19bbfeafb,2aae39f89f,3.894573,0.366814,0.306554,0.763325,0.000000,0.000000,...,0.350894,0.0,0.011453,0.000000,0.000000,0.000000,core,0,2.0,1.0
91,2021,1,a19bbfeafb,45b1ae2725,1.257788,0.366814,0.306554,0.763325,0.000000,0.000000,...,0.141496,0.0,0.043447,0.000000,0.000000,0.000000,super premium,0,2.0,1.0
92,2021,1,a19bbfeafb,53a67488ad,0.794904,0.366814,0.306554,0.763325,0.000000,0.000000,...,0.611176,0.0,0.019529,0.000000,0.000000,0.000000,core,0,2.0,1.0
93,2021,1,a19bbfeafb,85498197e2,11.651194,0.366814,0.306554,0.763325,0.097867,0.132583,...,0.279101,0.0,0.051353,0.000000,0.000000,0.000000,Unknown,0,2.0,1.0
94,2021,1,a19bbfeafb,967e3b26f8,18.567640,0.366814,0.306554,0.763325,0.036453,0.064803,...,0.351557,0.0,0.240463,0.000000,0.000000,0.000000,core,0,2.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
365,2024,2,aa30935544,71e9e5f0c0,1.515613,0.339480,0.319083,0.353705,0.000000,0.000000,...,0.993871,0.0,0.000007,0.001057,0.000499,0.161164,Unknown,13,1.0,0.0
366,2024,2,aa30935544,96ddb9088d,4.389797,0.339480,0.319083,0.353705,0.000692,0.032690,...,0.857280,0.0,0.001162,0.021892,0.008147,0.126964,Unknown,13,1.0,0.0
367,2024,2,aa30935544,Competitor,77.046284,0.339480,0.319083,0.353705,0.000000,0.000000,...,0.000000,0.0,0.000000,0.000000,0.000000,0.126968,Unknown,13,1.0,0.0
368,2024,2,aa30935544,a5f7e77993,1.725634,0.339480,0.319083,0.353705,0.000000,0.000000,...,0.992411,0.0,0.000005,0.000973,0.000451,0.158238,Unknown,13,1.0,0.0


In [111]:
print(df.columns)

Index(['year', 'quarter', 'country', 'brand', 'power', 'avg_temp',
       'avg_max_temp', 'avg_prcp', 'OOH', 'Other Digital Media',
       'Other Traditional Mktg', 'Radio', 'Social Media', 'TV', 'Trade',
       'all_other_marketing', 'contracts', 'total_spend', 'sales_vol', 'sales',
       'price', 'category', 'time_idx', 'quarter_sin', 'quarter_cos'],
      dtype='object')


In [46]:
df.to_csv('../created_data/preprocessed_data_full.csv', index=False)

In [126]:
months_for_test = 4
index_for_train_test_cut = df['time_idx'].max() - months_for_test
train_cut_off = df[df['time_idx'] <= index_for_train_test_cut].index.max()
print(train_cut_off)

329


In [113]:
# group id by country-year-month
df['group_id'] = df.groupby(['country', 'year', 'quarter']).ngroup()

group_id = df['group_id'].values
print(group_id)

[10 10 10 10 10 10 10 10 10 10 24 24 24 24 24 24 24 24 24 24 11 11 11 11
 11 11 11 11 11 11 25 25 25 25 25 25 25 25 25 25 12 12 12 12 12 12 12 12
 12 12 26 26 26 26 26 26 26 26 26 26 13 13 13 13 13 13 13 13 13 13 27 27
 27 27 27 27 27 27 27 27  0  0  0  0  0  0  0  0  0 14 14 14 14 14 14 14
 14 14 14 28 28 28 28 28 28 28 28 28 28  1  1  1  1  1  1  1  1  1 15 15
 15 15 15 15 15 15 15 15 29 29 29 29 29 29 29 29 29 29  2  2  2  2  2  2
  2  2  2 16 16 16 16 16 16 16 16 16 16 30 30 30 30 30 30 30 30 30 30  3
  3  3  3  3  3  3  3  3 17 17 17 17 17 17 17 17 17 17 31 31 31 31 31 31
 31 31 31 31  4  4  4  4  4  4  4  4  4 18 18 18 18 18 18 18 18 18 18 32
 32 32 32 32 32 32 32 32 32  5  5  5  5  5  5  5  5  5 19 19 19 19 19 19
 19 19 19 19 33 33 33 33 33 33 33 33 33 33  6  6  6  6  6  6  6  6  6 20
 20 20 20 20 20 20 20 20 20 34 34 34 34 34 34 34 34 34 34  7  7  7  7  7
  7  7  7  7 21 21 21 21 21 21 21 21 21 21 35 35 35 35 35 35 35 35 35 35
  8  8  8  8  8  8  8  8  8 22 22 22 22 22 22 22 22

In [114]:
# time_idx variable
time_idx = (df['time_idx'] + 1).values
print(time_idx)

# quarter sin and cos
quarter_sin = df['quarter_sin'].values
quarter_cos = df['quarter_cos'].values
print(quarter_sin)
print(quarter_cos)

[ 1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  2  2  2  2
  2  2  2  2  2  2  2  2  2  2  2  2  2  2  2  2  3  3  3  3  3  3  3  3
  3  3  3  3  3  3  3  3  3  3  3  3  4  4  4  4  4  4  4  4  4  4  4  4
  4  4  4  4  4  4  4  4  5  5  5  5  5  5  5  5  5  5  5  5  5  5  5  5
  5  5  5  5  5  5  5  5  5  5  5  5  5  6  6  6  6  6  6  6  6  6  6  6
  6  6  6  6  6  6  6  6  6  6  6  6  6  6  6  6  6  6  7  7  7  7  7  7
  7  7  7  7  7  7  7  7  7  7  7  7  7  7  7  7  7  7  7  7  7  7  7  8
  8  8  8  8  8  8  8  8  8  8  8  8  8  8  8  8  8  8  8  8  8  8  8  8
  8  8  8  8  9  9  9  9  9  9  9  9  9  9  9  9  9  9  9  9  9  9  9  9
  9  9  9  9  9  9  9  9  9 10 10 10 10 10 10 10 10 10 10 10 10 10 10 10
 10 10 10 10 10 10 10 10 10 10 10 10 10 10 11 11 11 11 11 11 11 11 11 11
 11 11 11 11 11 11 11 11 11 11 11 11 11 11 11 11 11 11 11 12 12 12 12 12
 12 12 12 12 12 12 12 12 12 12 12 12 12 12 12 12 12 12 12 12 12 12 12 12
 13 13 13 13 13 13 13 13 13 13 13 13 13 13 13 13 13

In [115]:
# country and brand categorical variables
country_id = df['country'].astype('category').cat.codes.values
brand_id = df['brand'].astype('category').cat.codes.values
print(country_id)
print(brand_id)

[1 1 1 1 1 1 1 1 1 1 2 2 2 2 2 2 2 2 2 2 1 1 1 1 1 1 1 1 1 1 2 2 2 2 2 2 2
 2 2 2 1 1 1 1 1 1 1 1 1 1 2 2 2 2 2 2 2 2 2 2 1 1 1 1 1 1 1 1 1 1 2 2 2 2
 2 2 2 2 2 2 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 2 2 2 2 2 2 2 2 2 2 0 0
 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 2 2 2 2 2 2 2 2 2 2 0 0 0 0 0 0 0 0 0 1
 1 1 1 1 1 1 1 1 1 2 2 2 2 2 2 2 2 2 2 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1
 1 2 2 2 2 2 2 2 2 2 2 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 2 2 2 2 2 2 2
 2 2 2 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 2 2 2 2 2 2 2 2 2 2 0 0 0 0 0
 0 0 0 0 1 1 1 1 1 1 1 1 1 1 2 2 2 2 2 2 2 2 2 2 0 0 0 0 0 0 0 0 0 1 1 1 1
 1 1 1 1 1 1 2 2 2 2 2 2 2 2 2 2 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 2 2
 2 2 2 2 2 2 2 2 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 2 2 2 2 2 2 2 2 2 2]
[ 5  6  8 13 15 17 18 20 21 26  2  4  7  9 10 11 16 17 19 23  5  6  8 13
 15 17 18 20 21 26  2  4  7  9 10 11 16 17 19 23  5  6  8 13 15 17 18 20
 21 26  2  4  7  9 10 11 16 17 19 23  5  6  8 13 15 17 18 20 21 26  2  4
  7  9 10 11 16 17 19 23  0  1

In [116]:
# vehicle categorical variables
vehicles = vehicle_cols
vehicle_id = pd.DataFrame({'vehicle': vehicles})['vehicle']
vehicle_id = vehicle_id.astype('category').cat.codes.values
print(vehicle_id)

[0 1 2 3 4 5 6 7 8]


In [117]:
# categorical variable - category
category_id = df['category'].astype('category').cat.codes.values
print(category_id)

[1 4 1 0 1 5 2 1 1 4 0 0 0 0 0 0 0 0 0 3 1 4 1 0 1 5 2 1 1 4 0 0 0 0 0 0 0
 0 0 3 1 4 1 0 1 5 2 1 1 4 0 0 0 0 0 0 0 0 0 3 1 4 1 0 1 5 2 1 1 4 0 0 0 0
 0 0 0 0 0 3 3 1 2 4 1 0 3 4 1 1 4 1 0 1 5 2 1 1 4 0 0 0 0 0 0 0 0 0 3 3 1
 2 4 1 0 3 4 1 1 4 1 0 1 5 2 1 1 4 0 0 0 0 0 0 0 0 0 3 3 1 2 4 1 0 3 4 1 1
 4 1 0 1 5 2 1 1 4 0 0 0 0 0 0 0 0 0 3 3 1 2 4 1 0 3 4 1 1 4 1 0 1 5 2 1 1
 4 0 0 0 0 0 0 0 0 0 3 3 1 2 4 1 0 3 4 1 1 4 1 0 1 5 2 1 1 4 0 0 0 0 0 0 0
 0 0 3 3 1 2 4 1 0 3 4 1 1 4 1 0 1 5 2 1 1 4 0 0 0 0 0 0 0 0 0 3 3 1 2 4 1
 0 3 4 1 1 4 1 0 1 5 2 1 1 4 0 0 0 0 0 0 0 0 0 3 3 1 2 4 1 0 3 4 1 1 4 1 0
 1 5 2 1 1 4 0 0 0 0 0 0 0 0 0 3 3 1 2 4 1 0 3 4 1 1 4 1 0 1 5 2 1 1 4 0 0
 0 0 0 0 0 0 0 3 3 1 2 4 1 0 3 4 1 1 4 1 0 1 5 2 1 1 4 0 0 0 0 0 0 0 0 0 3]


In [118]:
# price_controls
n_price_controls = 11
price_control_cols = [x for x in df.columns if x not in ['country', 'brand', 'year', 'quarter', 'power', 'time_idx', 'quarter_sin', 'quarter_cos', 'group_id', 'category'] \
                and x not in ['avg_temp', 'avg_max_temp', 'avg_prcp']\
                and x not in vehicles\
                and x not in ['total_spend']]
n_price_controls = len(price_control_cols)

price_control_ids = df[price_control_cols]


print(n_price_controls, price_control_cols, price_control_ids)
print()
print(np.array(price_control_cols).shape)

3 ['sales_vol', 'sales', 'price']      sales_vol     sales     price
90    0.000000  0.000000  0.000000
91    0.000000  0.000000  0.000000
92    0.000000  0.000000  0.000000
93    0.000000  0.000000  0.000000
94    0.000000  0.000000  0.000000
..         ...       ...       ...
365   0.001057  0.000499  0.161164
366   0.021892  0.008147  0.126964
367   0.000000  0.000000  0.126968
368   0.000973  0.000451  0.158238
369   0.000860  0.000350  0.138912

[370 rows x 3 columns]

(3,)


In [119]:
# vehicle controls
n_vehicle_controls = len(vehicles)
vehicle_control_ids = df[vehicles]
print(n_vehicle_controls, vehicles, vehicle_control_ids)
print()
print(np.array(vehicles).shape)

9 ['OOH', 'Other Digital Media', 'Other Traditional Mktg', 'Radio', 'Social Media', 'TV', 'Trade', 'all_other_marketing', 'contracts']           OOH  Other Digital Media  Other Traditional Mktg     Radio  \
90   0.000000             0.000000                0.000000  0.000000   
91   0.000000             0.000000                0.000000  0.000000   
92   0.000000             0.000000                0.000000  0.000000   
93   0.097867             0.132583                0.000000  0.036580   
94   0.036453             0.064803                0.000000  0.046807   
..        ...                  ...                     ...       ...   
365  0.000000             0.000000                0.001874  0.000000   
366  0.000692             0.032690                0.000221  0.000000   
367  0.000000             0.000000                0.000000  0.000000   
368  0.000000             0.000000                0.003112  0.000000   
369  0.079195             0.153498                0.000021  0.023481   



In [120]:
# macro controls
macro_control_cols = ['avg_temp', 'avg_max_temp', 'avg_prcp']
n_macro_controls = len(macro_control_cols)

macro_control_ids = df[macro_control_cols]
print(n_macro_controls, macro_control_cols, macro_control_ids)
print()
print(np.array(macro_control_cols).shape)

3 ['avg_temp', 'avg_max_temp', 'avg_prcp']      avg_temp  avg_max_temp  avg_prcp
90   0.366814      0.306554  0.763325
91   0.366814      0.306554  0.763325
92   0.366814      0.306554  0.763325
93   0.366814      0.306554  0.763325
94   0.366814      0.306554  0.763325
..        ...           ...       ...
365  0.339480      0.319083  0.353705
366  0.339480      0.319083  0.353705
367  0.339480      0.319083  0.353705
368  0.339480      0.319083  0.353705
369  0.339480      0.319083  0.353705

[370 rows x 3 columns]

(3,)


In [121]:
# total_spend
total_spend = df['total_spend'].values
print(total_spend)

[1.14532104e-02 4.34465967e-02 1.95291527e-02 5.13531826e-02
 2.40462735e-01 0.00000000e+00 1.09348625e-01 9.42313820e-02
 5.44176698e-01 1.09638631e-01 2.82758987e-03 3.49131715e-03
 1.35704395e-04 1.26979750e-04 1.03378133e-03 2.61929654e-05
 1.09245931e-03 0.00000000e+00 1.54706340e-05 1.42533027e-04
 1.26829809e-02 8.34182575e-02 1.12105832e-02 1.06684677e-01
 3.98802668e-01 0.00000000e+00 2.15216264e-01 6.67462647e-02
 4.50610816e-01 1.39005110e-01 2.70987325e-03 2.75051175e-03
 1.69221908e-04 1.38293835e-04 1.05268136e-03 2.96809703e-05
 1.34010718e-03 0.00000000e+00 2.42015903e-05 1.56231108e-04
 2.89623421e-02 5.86063787e-02 9.98526253e-03 2.33772784e-01
 2.19291478e-01 0.00000000e+00 2.69152224e-01 5.61112724e-02
 1.00000000e+00 2.28401914e-01 2.36370717e-03 3.08825052e-03
 1.79410170e-04 1.55322472e-04 9.04877379e-04 2.75325474e-05
 1.10063842e-03 0.00000000e+00 2.15310465e-05 1.41879442e-04
 1.20340828e-02 2.72790398e-02 2.16136631e-02 2.96963394e-01
 3.86440486e-01 0.000000

In [122]:
y_true = df['power'].values
print(y_true.shape)

(370,)


In [123]:
def fix_shape(arr, dtype=None):
    arr = np.array(arr)  # ensure ndarray
    arr = arr.reshape(arr.shape[0], -1)
    print(arr.shape)
    if arr.shape[0] < arr.shape[1]:
        arr = arr.T
    if dtype is not None:
        arr = arr.astype(dtype)
    print(arr.shape)
    print()
    return arr

time_idx   = fix_shape(time_idx, dtype="int32")
quarter_sin  = fix_shape(quarter_sin, dtype="float32")
quarter_cos  = fix_shape(quarter_cos, dtype="float32")
country_id = fix_shape(country_id, dtype="int32")
brand_id   = fix_shape(brand_id, dtype="int32")
vehicle_id = fix_shape(vehicle_id, dtype="int32")
category_id = fix_shape(category_id, dtype="int32")
price_controls = fix_shape(price_control_ids, dtype="float32")
vehicle_controls = fix_shape(vehicle_control_ids, dtype="float32")
macro_controls = fix_shape(macro_control_ids, dtype="float32")
total_spend = np.array(total_spend, dtype="float32").reshape(-1)   # (N,)
y_true     = np.array(y_true, dtype="float32").reshape(-1)   # (N,)
group_id   = np.array(group_id, dtype="int32").reshape(-1)   # (N,)

(370, 1)
(370, 1)

(370, 1)
(370, 1)

(370, 1)
(370, 1)

(370, 1)
(370, 1)

(370, 1)
(370, 1)

(9, 1)
(9, 1)

(370, 1)
(370, 1)

(370, 3)
(370, 3)

(370, 9)
(370, 9)

(370, 3)
(370, 3)



In [135]:
# train/test split
train_cut_off = len(y_true)
def train_test_split(arr, train_cut_off=train_cut_off):
    """Split the data into training and testing sets. 
    Input:
        arr: numpy array of (rows, cols)
        train_cut_off: index to split the data into training and testing sets

    Args:
        arr (np.ndarray): Input array to split.
        train_cut_off (int, optional): Index to split the data into training and testing sets. Defaults to train_cut_off.
    Returns:
        tuple: A tuple containing the training and testing sets for that array.
    """
    return arr[:train_cut_off+1], arr[train_cut_off+1:]

time_idx_train, time_idx_test = train_test_split(time_idx)
quarter_sin_train, quarter_sin_test = train_test_split(quarter_sin)
quarter_cos_train, quarter_cos_test = train_test_split(quarter_cos)
country_id_train, country_id_test = train_test_split(country_id)
brand_id_train, brand_id_test = train_test_split(brand_id)
vehicle_id_train, vehicle_id_test = train_test_split(vehicle_id)
category_id_train, category_id_test = train_test_split(category_id)
price_controls_train, price_controls_test = train_test_split(price_controls)
vehicle_controls_train, vehicle_controls_test = train_test_split(vehicle_controls)
macro_controls_train, macro_controls_test = train_test_split(macro_controls)
total_spend_train, total_spend_test = train_test_split(total_spend)
y_true_train, y_true_test = train_test_split(y_true)    
group_id_train, group_id_test = train_test_split(group_id)

print(time_idx_train.shape, time_idx_test.shape)

(370, 1) (0, 1)


In [128]:
# controls_test = np.zeros_like(controls_test)

# Pickle Output

In [136]:
# dump in a pickle file
output_path = "C:/Users/40107904/OneDrive - Anheuser-Busch InBev/ABI/WORK/hackathon_power/hackathon_lt_equity/dummy_data/processed_data"
output_file = f"{output_path}/preprocessed_data_submission_competitor_group.pkl"

input_data_dict = {
    "time_idx": time_idx_train,
    "quarter_sin": quarter_sin_train,
    "quarter_cos": quarter_cos_train,
    "country_id": country_id_train,
    "brand_id": brand_id_train,
    "vehicle_id": vehicle_id_train,
    "category_id": category_id_train,
    "price_controls": price_controls_train,
    "vehicle_controls": vehicle_controls_train,
    "macro_controls": macro_controls_train,
    "total_spend": total_spend_train,
    "y_true": y_true_train,
    "group_id": group_id_train
}

output_data_dict = {
    "time_idx": time_idx_test,
    "quarter_sin": quarter_sin_test,
    "quarter_cos": quarter_cos_test,
    "country_id": country_id_test,
    "brand_id": brand_id_test,
    "vehicle_id": vehicle_id_test,
    "category_id": category_id_test,
    "price_controls": price_controls_test,
    "vehicle_controls": vehicle_controls_test,
    "macro_controls": macro_controls_test,
    "total_spend": total_spend_test,
    "y_true": y_true_test,
    "group_id": group_id_test
}

final_dict = {
    "input_data": input_data_dict,
    "output_data": output_data_dict,
    "data": df
}

with open(output_file, "wb") as f:
    pkl.dump(final_dict, f)

In [133]:
df[train_cut_off+1:]

,year,quarter,country,brand,power,avg_temp,avg_max_temp,avg_prcp,OOH,Other Digital Media,...,contracts,total_spend,sales_vol,sales,price,category,time_idx,quarter_sin,quarter_cos,group_id
219,2024,1,a19bbfeafb,fdb6b2e750,14.630073,0.400623,0.355261,0.152954,0.048613,0.104472,...,0.000000,0.231002,0.112087,0.228187,0.694570,super premium,12,2.0,1.0,22
350,2024,1,aa30935544,1632ec491e,6.078193,0.047947,0.038530,0.331651,0.025416,0.087772,...,0.000000,0.003372,0.038687,0.014446,0.127396,Unknown,12,2.0,1.0,36
351,2024,1,aa30935544,2601ae05f5,3.195528,0.047947,0.038530,0.331651,0.021540,0.184000,...,0.000000,0.004172,0.034166,0.014998,0.149766,Unknown,12,2.0,1.0,36
352,2024,1,aa30935544,52be03db3b,1.092639,0.047947,0.038530,0.331651,0.000000,0.000000,...,0.000000,0.000058,0.012561,0.003569,0.096943,Unknown,12,2.0,1.0,36
353,2024,1,aa30935544,6c3e59bc24,1.741048,0.047947,0.038530,0.331651,0.000000,0.000000,...,0.000000,0.000027,0.005677,0.001727,0.103797,Unknown,12,2.0,1.0,36
354,2024,1,aa30935544,6e31ce004a,3.046745,0.047947,0.038530,0.331651,0.096482,0.118127,...,0.000000,0.000356,0.004066,0.002245,0.188395,Unknown,12,2.0,1.0,36
355,2024,1,aa30935544,71e9e5f0c0,1.346439,0.047947,0.038530,0.331651,0.000000,0.000000,...,0.000000,0.000006,0.000903,0.000434,0.163887,Unknown,12,2.0,1.0,36
356,2024,1,aa30935544,96ddb9088d,4.248669,0.047947,0.038530,0.331651,0.026131,0.047637,...,0.000000,0.000912,0.018513,0.006873,0.126653,Unknown,12,2.0,1.0,36
357,2024,1,aa30935544,Competitor,76.953724,0.047947,0.038530,0.331651,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.126968,Unknown,12,2.0,1.0,36
358,2024,1,aa30935544,a5f7e77993,1.537932,0.047947,0.038530,0.331651,0.000000,0.000000,...,0.000000,0.000005,0.000628,0.000298,0.162119,Unknown,12,2.0,1.0,36


In [134]:
df.loc[(df['country']=='aa30935544') & (df['brand']=='1632ec491e') ]

,year,quarter,country,brand,power,avg_temp,avg_max_temp,avg_prcp,OOH,Other Digital Media,...,contracts,total_spend,sales_vol,sales,price,category,time_idx,quarter_sin,quarter_cos,group_id
230,2021,1,aa30935544,1632ec491e,7.083773,0.006809,0.000000,0.220768,0.004402,0.032853,...,0.0,0.002828,0.063849,0.021288,0.113752,Unknown,0,2.0,1.0,24
240,2021,2,aa30935544,1632ec491e,6.639957,0.302065,0.289261,0.297021,0.007311,0.026437,...,0.0,0.002710,0.077537,0.026052,0.114632,Unknown,1,1.0,0.0,25
250,2021,3,aa30935544,1632ec491e,6.561854,0.444379,0.409719,0.397183,0.017143,0.033697,...,0.0,0.002364,0.075197,0.025184,0.114263,Unknown,2,0.0,1.0,26
260,2021,4,aa30935544,1632ec491e,6.790558,0.159195,0.149140,0.198305,0.011061,0.011598,...,0.0,0.002799,0.068377,0.023005,0.114785,Unknown,3,1.0,2.0,27
270,2022,1,aa30935544,1632ec491e,6.665683,0.000000,0.011052,0.206507,0.015972,0.013925,...,0.0,0.001750,0.059843,0.020809,0.118636,Unknown,4,2.0,1.0,28
280,2022,2,aa30935544,1632ec491e,6.612967,0.303922,0.287663,0.296769,0.016147,0.019318,...,0.0,0.001374,0.070111,0.024607,0.119745,Unknown,5,1.0,0.0,29
290,2022,3,aa30935544,1632ec491e,6.580702,0.451644,0.419637,0.297293,0.006569,0.016455,...,0.0,0.001713,0.071333,0.024939,0.119281,Unknown,6,0.0,1.0,30
300,2022,4,aa30935544,1632ec491e,6.318342,0.110573,0.104001,0.216201,0.008712,0.069276,...,0.0,0.003819,0.066021,0.024409,0.126140,Unknown,7,1.0,2.0,31
310,2023,1,aa30935544,1632ec491e,6.420912,0.041589,0.029787,0.237920,0.023685,0.101267,...,0.0,0.003507,0.050986,0.018896,0.126447,Unknown,8,2.0,1.0,32
320,2023,2,aa30935544,1632ec491e,5.922186,0.292464,0.279552,0.230576,0.039391,0.138607,...,0.0,0.002505,0.051782,0.018981,0.125060,Unknown,9,1.0,0.0,33


# To create test dicts

In [137]:
df_test = df.loc[((df['year'] == 2023) & df['quarter'].isin([3, 4])) | ((df['year'] == 2024) & df['quarter'].isin([1, 2]))].reset_index(drop=True)
df_test

,year,quarter,country,brand,power,avg_temp,avg_max_temp,avg_prcp,OOH,Other Digital Media,...,contracts,total_spend,sales_vol,sales,price,category,time_idx,quarter_sin,quarter_cos,group_id
0,2023,3,653b4e6d12,05e5aaf0f4,2.771549,0.436275,0.379421,0.000000,0.000000,0.000000,...,0.353467,0.000415,0.021788,0.047700,0.746919,premium,10,0.0,1.0,6
1,2023,3,653b4e6d12,0ec35f4c94,3.291818,0.436275,0.379421,0.000000,0.000000,0.000000,...,0.533306,0.000532,0.051959,0.075773,0.497539,core,10,0.0,1.0,6
2,2023,3,653b4e6d12,2441ad3052,4.974649,0.436275,0.379421,0.000000,0.008736,0.250030,...,0.111476,0.000704,0.012799,0.023758,0.633301,core+,10,0.0,1.0,6
3,2023,3,653b4e6d12,7690b7ccb9,3.093721,0.436275,0.379421,0.000000,0.255167,0.129077,...,0.014760,0.001756,0.002341,0.006862,1.000000,super premium,10,0.0,1.0,6
4,2023,3,653b4e6d12,875962380f,11.364056,0.436275,0.379421,0.000000,0.050036,0.062980,...,0.339840,0.001414,0.093781,0.137762,0.501178,core,10,0.0,1.0,6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
111,2024,2,aa30935544,71e9e5f0c0,1.515613,0.339480,0.319083,0.353705,0.000000,0.000000,...,0.000000,0.000007,0.001057,0.000499,0.161164,Unknown,13,1.0,0.0,37
112,2024,2,aa30935544,96ddb9088d,4.389797,0.339480,0.319083,0.353705,0.000692,0.032690,...,0.000000,0.001162,0.021892,0.008147,0.126964,Unknown,13,1.0,0.0,37
113,2024,2,aa30935544,Competitor,77.046284,0.339480,0.319083,0.353705,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.126968,Unknown,13,1.0,0.0,37
114,2024,2,aa30935544,a5f7e77993,1.725634,0.339480,0.319083,0.353705,0.000000,0.000000,...,0.000000,0.000005,0.000973,0.000451,0.158238,Unknown,13,1.0,0.0,37


In [138]:
df_test['year'] += 1
df_test['time_idx'] += 4
df_test

,year,quarter,country,brand,power,avg_temp,avg_max_temp,avg_prcp,OOH,Other Digital Media,...,contracts,total_spend,sales_vol,sales,price,category,time_idx,quarter_sin,quarter_cos,group_id
0,2024,3,653b4e6d12,05e5aaf0f4,2.771549,0.436275,0.379421,0.000000,0.000000,0.000000,...,0.353467,0.000415,0.021788,0.047700,0.746919,premium,14,0.0,1.0,6
1,2024,3,653b4e6d12,0ec35f4c94,3.291818,0.436275,0.379421,0.000000,0.000000,0.000000,...,0.533306,0.000532,0.051959,0.075773,0.497539,core,14,0.0,1.0,6
2,2024,3,653b4e6d12,2441ad3052,4.974649,0.436275,0.379421,0.000000,0.008736,0.250030,...,0.111476,0.000704,0.012799,0.023758,0.633301,core+,14,0.0,1.0,6
3,2024,3,653b4e6d12,7690b7ccb9,3.093721,0.436275,0.379421,0.000000,0.255167,0.129077,...,0.014760,0.001756,0.002341,0.006862,1.000000,super premium,14,0.0,1.0,6
4,2024,3,653b4e6d12,875962380f,11.364056,0.436275,0.379421,0.000000,0.050036,0.062980,...,0.339840,0.001414,0.093781,0.137762,0.501178,core,14,0.0,1.0,6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
111,2025,2,aa30935544,71e9e5f0c0,1.515613,0.339480,0.319083,0.353705,0.000000,0.000000,...,0.000000,0.000007,0.001057,0.000499,0.161164,Unknown,17,1.0,0.0,37
112,2025,2,aa30935544,96ddb9088d,4.389797,0.339480,0.319083,0.353705,0.000692,0.032690,...,0.000000,0.001162,0.021892,0.008147,0.126964,Unknown,17,1.0,0.0,37
113,2025,2,aa30935544,Competitor,77.046284,0.339480,0.319083,0.353705,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.126968,Unknown,17,1.0,0.0,37
114,2025,2,aa30935544,a5f7e77993,1.725634,0.339480,0.319083,0.353705,0.000000,0.000000,...,0.000000,0.000005,0.000973,0.000451,0.158238,Unknown,17,1.0,0.0,37


In [65]:
# output = df_test.copy()

In [139]:
output_format_path = r'C:\Users\40107904\OneDrive - Anheuser-Busch InBev\ABI\WORK\hackathon_power\hackathon_lt_equity\data\lte_participants_data\BG_Data_Hackathon_Test_Predict_masked.csv'
output = pd.read_csv(output_format_path)
output.columns = output.columns.str.lower().str.strip()
output['quarter'] = output['quarter'].apply(lambda x: int(x[-1: ]))

output = output.merge(df_test, on=['country', 'brand', 'year', 'quarter'], how='outer')
output.drop(columns=['predicted power'], inplace=True)

output

,year,quarter,country,brand,power,avg_temp,avg_max_temp,avg_prcp,OOH,Other Digital Media,...,contracts,total_spend,sales_vol,sales,price,category,time_idx,quarter_sin,quarter_cos,group_id
0,2024,3,653b4e6d12,05e5aaf0f4,2.771549,0.436275,0.379421,0.000000,0.000000,0.000000,...,0.353467,0.000415,0.021788,0.047700,0.746919,premium,14.0,0.0,1.0,6.0
1,2024,4,653b4e6d12,05e5aaf0f4,2.922052,0.502078,0.434113,0.268535,0.000000,0.000000,...,0.386159,0.000361,0.025594,0.056429,0.752209,premium,15.0,1.0,2.0,7.0
2,2025,1,653b4e6d12,05e5aaf0f4,3.118191,0.488990,0.400646,0.722165,0.000000,0.000000,...,0.497212,0.000283,0.028001,0.060925,0.742334,premium,16.0,2.0,1.0,8.0
3,2025,2,653b4e6d12,05e5aaf0f4,3.235267,0.443439,0.372093,0.166700,0.000000,0.000000,...,0.470411,0.000289,0.025798,0.056685,0.749653,premium,17.0,1.0,0.0,9.0
4,2024,3,653b4e6d12,0ec35f4c94,3.291818,0.436275,0.379421,0.000000,0.000000,0.000000,...,0.533306,0.000532,0.051959,0.075773,0.497539,core,14.0,0.0,1.0,6.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
115,2025,2,aa30935544,a5f7e77993,1.725634,0.339480,0.319083,0.353705,0.000000,0.000000,...,0.000000,0.000005,0.000973,0.000451,0.158238,Unknown,17.0,1.0,0.0,37.0
116,2024,3,aa30935544,e7286b8344,0.669829,0.444197,0.411562,0.167785,0.132191,0.157924,...,0.000000,0.000318,0.001016,0.000397,0.133265,premium,14.0,0.0,1.0,34.0
117,2024,4,aa30935544,e7286b8344,0.698816,0.134305,0.117682,0.219971,0.048496,0.048339,...,0.000000,0.000132,0.000709,0.000280,0.134670,premium,15.0,1.0,2.0,35.0
118,2025,1,aa30935544,e7286b8344,0.759083,0.047947,0.038530,0.331651,0.023008,0.017843,...,0.000000,0.000141,0.000582,0.000234,0.137366,premium,16.0,2.0,1.0,36.0


In [140]:
output[output.isna().any(axis=1)]
# for these rows, just place avg values for each column within the same country
# output = output.fillna(output.groupby(['country', 'brand', 'year', 'quarter']).transform('mean'))

,year,quarter,country,brand,power,avg_temp,avg_max_temp,avg_prcp,OOH,Other Digital Media,...,contracts,total_spend,sales_vol,sales,price,category,time_idx,quarter_sin,quarter_cos,group_id
48,2024,3,a19bbfeafb,8.55E+09,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
49,2024,4,a19bbfeafb,8.55E+09,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
50,2025,1,a19bbfeafb,8.55E+09,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
51,2025,2,a19bbfeafb,8.55E+09,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [141]:
numeric_cols = []
for col in output.select_dtypes(include=['float64', 'int64', 'float32', 'int32']).columns:
    if col not in ['year', 'quarter']:
        numeric_cols.append(col)
fill_value = output.loc[output['country'] == 'a19bbfeafb'][['country', 'year', 'quarter']+numeric_cols].groupby(['country', 'year', 'quarter']).transform('mean')

In [142]:
output = output.fillna(fill_value) 
output

,year,quarter,country,brand,power,avg_temp,avg_max_temp,avg_prcp,OOH,Other Digital Media,...,contracts,total_spend,sales_vol,sales,price,category,time_idx,quarter_sin,quarter_cos,group_id
0,2024,3,653b4e6d12,05e5aaf0f4,2.771549,0.436275,0.379421,0.000000,0.000000,0.000000,...,0.353467,0.000415,0.021788,0.047700,0.746919,premium,14.0,0.0,1.0,6.0
1,2024,4,653b4e6d12,05e5aaf0f4,2.922052,0.502078,0.434113,0.268535,0.000000,0.000000,...,0.386159,0.000361,0.025594,0.056429,0.752209,premium,15.0,1.0,2.0,7.0
2,2025,1,653b4e6d12,05e5aaf0f4,3.118191,0.488990,0.400646,0.722165,0.000000,0.000000,...,0.497212,0.000283,0.028001,0.060925,0.742334,premium,16.0,2.0,1.0,8.0
3,2025,2,653b4e6d12,05e5aaf0f4,3.235267,0.443439,0.372093,0.166700,0.000000,0.000000,...,0.470411,0.000289,0.025798,0.056685,0.749653,premium,17.0,1.0,0.0,9.0
4,2024,3,653b4e6d12,0ec35f4c94,3.291818,0.436275,0.379421,0.000000,0.000000,0.000000,...,0.533306,0.000532,0.051959,0.075773,0.497539,core,14.0,0.0,1.0,6.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
115,2025,2,aa30935544,a5f7e77993,1.725634,0.339480,0.319083,0.353705,0.000000,0.000000,...,0.000000,0.000005,0.000973,0.000451,0.158238,Unknown,17.0,1.0,0.0,37.0
116,2024,3,aa30935544,e7286b8344,0.669829,0.444197,0.411562,0.167785,0.132191,0.157924,...,0.000000,0.000318,0.001016,0.000397,0.133265,premium,14.0,0.0,1.0,34.0
117,2024,4,aa30935544,e7286b8344,0.698816,0.134305,0.117682,0.219971,0.048496,0.048339,...,0.000000,0.000132,0.000709,0.000280,0.134670,premium,15.0,1.0,2.0,35.0
118,2025,1,aa30935544,e7286b8344,0.759083,0.047947,0.038530,0.331651,0.023008,0.017843,...,0.000000,0.000141,0.000582,0.000234,0.137366,premium,16.0,2.0,1.0,36.0


In [143]:
df['country_id'] = country_id_train
df['brand_id'] = brand_id_train
df['category_id'] = category_id_train
df.reset_index(drop=True, inplace=True)

country_brand_id_map = df[['country', 'brand', 'country_id', 'brand_id', 'category_id']].drop_duplicates().reset_index(drop=True)

country_brand_id_map

,country,brand,country_id,brand_id,category_id
0,a19bbfeafb,2aae39f89f,1,5,1
1,a19bbfeafb,45b1ae2725,1,6,4
2,a19bbfeafb,53a67488ad,1,8,1
3,a19bbfeafb,85498197e2,1,13,0
4,a19bbfeafb,967e3b26f8,1,15,1
5,a19bbfeafb,Competitor,1,17,5
6,a19bbfeafb,a0bc54a71d,1,18,2
7,a19bbfeafb,b6a118bf97,1,20,1
8,a19bbfeafb,cd46a210c6,1,21,1
9,a19bbfeafb,fdb6b2e750,1,26,4


In [144]:
output = output.merge(country_brand_id_map, on=['country', 'brand'], how='left')
output['country_id'] = output['country_id'].fillna(1.0)
output['brand_id'] = output['brand_id'].fillna(27.0)
output['category_id'] = output['category_id'].fillna(0.0)
output['category'] = output['category'].fillna('Unknown')
output

,year,quarter,country,brand,power,avg_temp,avg_max_temp,avg_prcp,OOH,Other Digital Media,...,sales,price,category,time_idx,quarter_sin,quarter_cos,group_id,country_id,brand_id,category_id
0,2024,3,653b4e6d12,05e5aaf0f4,2.771549,0.436275,0.379421,0.000000,0.000000,0.000000,...,0.047700,0.746919,premium,14.0,0.0,1.0,6.0,0.0,0.0,3.0
1,2024,4,653b4e6d12,05e5aaf0f4,2.922052,0.502078,0.434113,0.268535,0.000000,0.000000,...,0.056429,0.752209,premium,15.0,1.0,2.0,7.0,0.0,0.0,3.0
2,2025,1,653b4e6d12,05e5aaf0f4,3.118191,0.488990,0.400646,0.722165,0.000000,0.000000,...,0.060925,0.742334,premium,16.0,2.0,1.0,8.0,0.0,0.0,3.0
3,2025,2,653b4e6d12,05e5aaf0f4,3.235267,0.443439,0.372093,0.166700,0.000000,0.000000,...,0.056685,0.749653,premium,17.0,1.0,0.0,9.0,0.0,0.0,3.0
4,2024,3,653b4e6d12,0ec35f4c94,3.291818,0.436275,0.379421,0.000000,0.000000,0.000000,...,0.075773,0.497539,core,14.0,0.0,1.0,6.0,0.0,1.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
115,2025,2,aa30935544,a5f7e77993,1.725634,0.339480,0.319083,0.353705,0.000000,0.000000,...,0.000451,0.158238,Unknown,17.0,1.0,0.0,37.0,2.0,19.0,0.0
116,2024,3,aa30935544,e7286b8344,0.669829,0.444197,0.411562,0.167785,0.132191,0.157924,...,0.000397,0.133265,premium,14.0,0.0,1.0,34.0,2.0,23.0,3.0
117,2024,4,aa30935544,e7286b8344,0.698816,0.134305,0.117682,0.219971,0.048496,0.048339,...,0.000280,0.134670,premium,15.0,1.0,2.0,35.0,2.0,23.0,3.0
118,2025,1,aa30935544,e7286b8344,0.759083,0.047947,0.038530,0.331651,0.023008,0.017843,...,0.000234,0.137366,premium,16.0,2.0,1.0,36.0,2.0,23.0,3.0


In [145]:
for col in numeric_cols:
    if col in output.columns and col not in ['year', 'quarter', 'power', 'time_idx', 'quarter_sin', 'quarter_cos', 'country_id', 'brand_id', 'category_id', 'total_spend', 'group_id'] and col not in vehicle_cols:
        output[col] += np.random.normal(0, 0.1, output.shape[0]) - 0.05

In [146]:
output['total_spend'] *= 1.1

In [147]:
time_idx_test = (output['time_idx'] + 1).values
quarter_sin_test = output['quarter_sin'].values
quarter_cos_test = output['quarter_cos'].values
country_id_test = output['country_id'].astype('int32').values
brand_id_test = output['brand_id'].astype('int32').values
category_id_test = output['category_id'].astype('int32').values
price_controls_test = output[price_control_cols].values.astype('float32')
vehicle_controls_test = output[vehicles].values.astype('float32')
macro_controls_test = output[macro_control_cols].values.astype('float32')
total_spend_test = output['total_spend'].values.astype('float32')
y_true_test = output['power'].values.astype('float32')
group_id_test = output['group_id'].astype('int32').values

test_dict = {
    "time_idx": time_idx_test,
    "quarter_sin": quarter_sin_test,
    "quarter_cos": quarter_cos_test,
    "country_id": country_id_test,
    "brand_id": brand_id_test,
    "category_id": category_id_test,
    # "vehicle_id": vehicle_id_test,
    "price_controls": price_controls_test,
    "vehicle_controls": vehicle_controls_test,
    "macro_controls": macro_controls_test,
    "total_spend": total_spend_test,
    "y_true": y_true_test,
    "group_id": group_id_test,
    "df_reference": output
}

In [148]:
output_path = "C:/Users/40107904/OneDrive - Anheuser-Busch InBev/ABI/WORK/hackathon_power/hackathon_lt_equity/dummy_data/processed_data"
with open(f"{output_path}/preprocessed_data_submission_format_competitor_group.pkl", "wb") as f:
    pkl.dump(test_dict, f)